# ETL with Spark

This notebook serves as a testing ground for the Spark ETL job for the script `src/data/etl_spark.py`.

In [1]:
## IMPORTS
# PYSPARK
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import to_timestamp, countDistinct, to_date

# MISCELLANEOUS
from sys import path # import helper_functions.py
path.insert(1, '../src/data') # insert at 1, 0 is the script path (or '' in REPL)
import helper_functions as h

## Explore core specifications

In [2]:
# create session
spark = SparkSession\
    .builder \
    .appName("ETL for esg") \
    .getOrCreate()

print("Spark context:\n")
print(*spark.sparkContext.getConf().getAll(), sep='\n')

# read in first gtrends files
# preprocessed query output
path_input = h.get_files('../data/raw', name_contains="*gtrends*preprocessed*")[0]
df_raw = spark.read.csv(path_input, header=True)
# query input
path_input = h.get_files('../data/raw', name_contains="*gtrends*meta*")[0]
df_raw_meta = spark.read.csv(path_input, header=True)

print('-'*40)
print(f"Load file:{path_input}")


# print description
print('-'*40)
print(df_raw.printSchema())
print('-'*40)
df_nrows = df_raw.count()
df_ncols = len(df_raw.columns)
print("Rows={}, Columns={}".format(df_nrows, df_ncols))
print('-'*40)
print("METADATA")
print(df_raw_meta.printSchema())
print('-'*40)
df_meta_nrows = df_raw_meta.count()
df_meta_ncols = len(df_raw_meta.columns)
print("Rows={}, Columns={}".format(df_meta_nrows, df_meta_ncols))

Spark context:

('spark.driver.host', 'host.docker.internal')
('spark.app.id', 'local-1604562321954')
('spark.rdd.compress', 'True')
('spark.app.name', 'ETL for esg')
('spark.serializer.objectStreamReset', '100')
('spark.master', 'local[*]')
('spark.submit.pyFiles', '')
('spark.executor.id', 'driver')
('spark.submit.deployMode', 'client')
('spark.driver.port', '57250')
('spark.ui.showConsoleProgress', 'true')
----------------------------------------
Load file:C:\Users\Philipp\GDrive\Projekter\Courses\Udacity Data Engineer\5 Capstone Project\data_engineer_capstone\data\raw\20201017-191627gtrends_metadata.csv
----------------------------------------
root
 |-- date: string (nullable = true)
 |-- keyword: string (nullable = true)
 |-- search_interest: string (nullable = true)

None
----------------------------------------
Rows=998325, Columns=3
----------------------------------------
METADATA
root
 |-- topic: string (nullable = true)
 |-- positive: string (nullable = true)
 |-- date_defin

In [3]:
spark

# Preprocessing

1. rename columns
2. set column types (date, string, int)

## Gtrends

In [4]:
# rename columns
df_renamed = df_raw.withColumnRenamed("_c0","date")\
    .withColumnRenamed("_c1","keyword")\
    .withColumnRenamed("_c2","search_interest")

# cast column types
df = df_renamed.withColumn("search_interest", df_renamed["search_interest"].cast(IntegerType()))\
    .withColumn('date', to_date(df_renamed.date))

## Metadata (API input)

1. convert date strings to date type

In [5]:
# change date string to date type
df_meta = df_raw_meta.withColumn('date_define_topic', to_date(df_raw_meta.date_define_topic))\
    .withColumn('date_get_firmname', to_date(df_raw_meta.date_get_firmname))\
    .withColumn('date_construct_keyword', to_date(df_raw_meta.date_construct_keyword))\
    .withColumn('date_query_googletrends', to_date(df_raw_meta.date_query_googletrends))

# Validation

Validation is done with `Spark SQL` due to learning purposes.

To validate the dataset, the input data from `metadata` can be compared to the API output `gtrends`. 



In [6]:
# create temporary view for SQL
df.createOrReplaceTempView("df")
df_meta.createOrReplaceTempView("meta")

#### Distinct counts of keyword and date

In [7]:
# count distinct keywords, dates and 
kw_count = spark.sql("""
    SELECT COUNT(DISTINCT keyword), COUNT(DISTINCT date)
    FROM df
    """).collect()

distinct_kw_date = [i for i in kw_count[0]]
print("\tDISTINCT \nkeywords \tdates \t=dates*keywords")
print("-"*40)
print("{}\t\t {}\t {}".format(distinct_kw_date[0], distinct_kw_date[1], distinct_kw_date[0]*distinct_kw_date[1]))

	DISTINCT 
keywords 	dates 	=dates*keywords
----------------------------------------
3825		 261	 998325


#### Set difference Input//Output

Store delta between input and output as `failed_queries`

In [99]:
# get set difference of meta (input)/df (output)
set_difference_keywords = spark.sql("""
    SELECT DISTINCT in.keyword
    FROM meta AS in
    WHERE in.keyword NOT IN (
        SELECT DISTINCT out.keyword 
        FROM df AS out)
""")

In [120]:
# get date range of queries and add to metadata
# most recent date record
date_first = spark.sql("""
    SELECT DISTINCT date
    FROM df
    ORDER BY date DESC
    LIMIT 1
""").collect ()

# first date record
date_last = spark.sql("""
    SELECT DISTINCT date
    FROM df
    ORDER BY date ASC
    LIMIT 1
""").collect()

print("First (earliest) date record: {}".format(date_first[0].date))
print("Last (latest) date record: {}".format(date_last[0].date))

First (earliest) date record: 2020-10-11
Last (latest) date record: 2015-10-18


# Export in parquet to S3

The previous steps explored and validated the query input and output data. We arrive at the following datasets which are exported in parquet format to AWS S3.  

1. Query output: Google Trends search interest
2. Query input: Google Trends meta data
3. Query failure: Not every query resulted in  Difference between (1) query input and (2) query output

In [121]:
print(set_difference_keywords.show())

+--------------------+
|             keyword|
+--------------------+
|hate Alexion Phar...|
|issue American In...|
|problem Alexion P...|
|scam American Int...|
|bad Alexion Pharm...|
|strike American I...|
|controversial Ame...|
|lawsuit Alexion P...|
|trouble American ...|
|unfair Alexion Ph...|
+--------------------+

None


In [119]:
spark.conf.set("spark.sql.parquet.compression.codec", "snappy")
df_meta.write.save("./query_metadata.parquet", format="parquet", mode='overwrite')
set_difference_keywords.write.save("./query_metadata.parquet", format="parquet", mode='overwrite')
df.write.parquet("s3://esg-analytics/processed_gtrends.parquet", mode='overwrite')

Py4JJavaError: An error occurred while calling o444.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:226)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:175)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:213)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:210)
	at o

In [ ]:
# store in parquet
# df.write.save("namesAndAges.parquet", format="parquet")

df.write.parquet("output/proto.parquet")
df.write.parquet("s3://sparkbyexamples/parquet/people.parquet")
# TODO: Store on S3

# Spark Script Example: Random text classification

In [ ]:
# pyspark
import argparse

from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import array_contains


def random_text_classifier(input_loc, output_loc):
    """
    This is a dummy function to show how to use spark, It is supposed to mock
    the following steps
        1. clean input data
        2. use a pre-trained model to make prediction 
        3. write predictions to a HDFS output

    Since this is meant as an example, we are going to skip building a model,
    instead we are naively going to mark reviews having the text "good" as positive and
    the rest as negative 
    """

    # read input
    df_raw = spark.read.option("header", True).csv(input_loc)
    # perform text cleaning

    # Tokenize text
    tokenizer = Tokenizer(inputCol="review_str", outputCol="review_token")
    df_tokens = tokenizer.transform(df_raw).select("cid", "review_token")

    # Remove stop words
    remover = StopWordsRemover(inputCol="review_token", outputCol="review_clean")
    df_clean = remover.transform(df_tokens).select("cid", "review_clean")

    # function to check presence of good
    df_out = df_clean.select(
        "cid", array_contains(df_clean.review_clean, "good").alias("positive_review")
    )
    # parquet is a popular column storage format, we use it here
    df_out.write.mode("overwrite").parquet(output_loc)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--input", type=str, help="HDFS input", default="/movie")
    parser.add_argument("--output", type=str, help="HDFS output", default="/output")
    args = parser.parse_args()
    spark = SparkSession.builder.appName("Random Text Classifier").getOrCreate()
    random_text_classifier(input_loc=args.input, output_loc=args.output)
